In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objs as go
import datetime
import numpy as np

Load data from CSV

In [2]:
dataframe = pd.read_csv('data.csv')
dataframe

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,TV Show,3%,NaN,"João Miguel, Bianca Comparato, Michel Gomes, R...",Brazil,"August 14, 2020",2020,TV-MA,4 Seasons,"International TV Shows, TV Dramas, TV Sci-Fi &...",In a future where the elite inhabit an island ...
1,s2,Movie,7:19,Jorge Michel Grau,"Demián Bichir, Héctor Bonilla, Oscar Serrano, ...",Mexico,"December 23, 2016",2016,TV-MA,93 min,"Dramas, International Movies",After a devastating earthquake hits Mexico Cit...
2,s3,Movie,23:59,Gilbert Chan,"Tedd Chan, Stella Chung, Henley Hii, Lawrence ...",Singapore,"December 20, 2018",2011,R,78 min,"Horror Movies, International Movies","When an army recruit is found dead, his fellow..."
3,s4,Movie,9,Shane Acker,"Elijah Wood, John C. Reilly, Jennifer Connelly...",United States,"November 16, 2017",2009,PG-13,80 min,"Action & Adventure, Independent Movies, Sci-Fi...","In a postapocalyptic world, rag-doll robots hi..."
4,s5,Movie,21,Robert Luketic,"Jim Sturgess, Kevin Spacey, Kate Bosworth, Aar...",United States,"January 1, 2020",2008,PG-13,123 min,Dramas,A brilliant group of students become card-coun...
...,...,...,...,...,...,...,...,...,...,...,...,...
7782,s7783,Movie,Zozo,Josef Fares,"Imad Creidi, Antoinette Turk, Elias Gergi, Car...","Sweden, Czech Republic, United Kingdom, Denmar...","October 19, 2020",2005,TV-MA,99 min,"Dramas, International Movies",When Lebanon's Civil War deprives Zozo of his ...
7783,s7784,Movie,Zubaan,Mozez Singh,"Vicky Kaushal, Sarah-Jane Dias, Raaghav Chanan...",India,"March 2, 2019",2015,TV-14,111 min,"Dramas, International Movies, Music & Musicals",A scrappy but poor boy worms his way into a ty...
7784,s7785,Movie,Zulu Man in Japan,NaN,Nasty C,NaN,"September 25, 2020",2019,TV-MA,44 min,"Documentaries, International Movies, Music & M...","In this documentary, South African rapper Nast..."
7785,s7786,TV Show,Zumbo's Just Desserts,NaN,"Adriano Zumbo, Rachel Khoo",Australia,"October 31, 2020",2019,TV-PG,1 Season,"International TV Shows, Reality TV",Dessert wizard Adriano Zumbo looks for the nex...


### 1. Number of titles produced by each country added yearly on Netflix

We observe that some shows have multiple countries separated by `,`

In [3]:
dataframe[dataframe['country'].str.contains(",",na=False)]

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
11,s12,TV Show,1983,NaN,"Robert Więckiewicz, Maciej Musiał, Michalina O...","Poland, United States","November 30, 2018",2018,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Dramas","In this dark alt-history thriller, a naïve law..."
17,s18,Movie,22-Jul,Paul Greengrass,"Anders Danielsen Lie, Jon Øigarden, Jonas Stra...","Norway, Iceland, United States","October 10, 2018",2018,R,144 min,"Dramas, Thrillers","After devastating terror attacks in Norway, a ..."
48,s49,Movie,10 Days in Sun City,Adze Ugah,"Ayo Makun, Adesua Etomi, Richard Mofe-Damijo, ...","South Africa, Nigeria","October 18, 2019",2017,TV-14,87 min,"Comedies, International Movies, Romantic Movies",After his girlfriend wins the Miss Nigeria pag...
50,s51,Movie,"10,000 B.C.",Roland Emmerich,"Steven Strait, Camilla Belle, Cliff Curtis, Jo...","United States, South Africa","June 1, 2019",2008,PG-13,109 min,Action & Adventure,Fierce mammoth hunter D'Leh sets out on an imp...
54,s55,Movie,100 Meters,Marcel Barrena,"Dani Rovira, Karra Elejalde, Alexandra Jiménez...","Portugal, Spain","March 10, 2017",2016,TV-MA,109 min,"Dramas, International Movies, Sports Movies",A man who is diagnosed with multiple sclerosis...
...,...,...,...,...,...,...,...,...,...,...,...,...
7759,s7760,TV Show,Zak Storm,NaN,"Michael Johnston, Jessica Gee-George, Christin...","United States, France, South Korea, Indonesia","September 13, 2018",2016,TV-Y7,3 Seasons,Kids' TV,Teen surfer Zak Storm is mysteriously transpor...
7761,s7762,Movie,Zapped,Peter DeLuise,"Zendaya, Chanelle Peloso, Spencer Boldman, Emi...","Canada, United States","February 1, 2017",2014,TV-Y,92 min,"Children & Family Movies, Comedies",A girl discovers a dog-training app that can g...
7770,s7771,Movie,Zinzana,Majid Al Ansari,"Ali Suliman, Saleh Bakri, Yasa, Ali Al-Jabri, ...","United Arab Emirates, Jordan","March 9, 2016",2015,TV-MA,96 min,"Dramas, International Movies, Thrillers",Recovering alcoholic Talal wakes up inside a s...
7782,s7783,Movie,Zozo,Josef Fares,"Imad Creidi, Antoinette Turk, Elias Gergi, Car...","Sweden, Czech Republic, United Kingdom, Denmar...","October 19, 2020",2005,TV-MA,99 min,"Dramas, International Movies",When Lebanon's Civil War deprives Zozo of his ...


Let's try to create arrays from the comma separated country values.

In [4]:
by_country = dataframe[['title', 'country', 'date_added',
                        'release_year']].copy()
by_country['country'] = by_country['country'].apply(lambda row_value: list(
    map(lambda country_name: country_name.strip(), str(row_value).split(","))))
by_country

,title,country,date_added,release_year
0,3%,[Brazil],"August 14, 2020",2020
1,7:19,[Mexico],"December 23, 2016",2016
2,23:59,[Singapore],"December 20, 2018",2011
3,9,[United States],"November 16, 2017",2009
4,21,[United States],"January 1, 2020",2008
...,...,...,...,...
7782,Zozo,"[Sweden, Czech Republic, United Kingdom, Denma...","October 19, 2020",2005
7783,Zubaan,[India],"March 2, 2019",2015
7784,Zulu Man in Japan,[nan],"September 25, 2020",2019
7785,Zumbo's Just Desserts,[Australia],"October 31, 2020",2019


Now we can use the `explode` method to expand the records which have multiple countries. 

In [5]:
expanded_countries = by_country.explode('country')

expanded_countries

,title,country,date_added,release_year
0,3%,Brazil,"August 14, 2020",2020
1,7:19,Mexico,"December 23, 2016",2016
2,23:59,Singapore,"December 20, 2018",2011
3,9,United States,"November 16, 2017",2009
4,21,United States,"January 1, 2020",2008
...,...,...,...,...
7784,Zulu Man in Japan,nan,"September 25, 2020",2019
7785,Zumbo's Just Desserts,Australia,"October 31, 2020",2019
7786,ZZ TOP: THAT LITTLE OL' BAND FROM TEXAS,United Kingdom,"March 1, 2020",2019
7786,ZZ TOP: THAT LITTLE OL' BAND FROM TEXAS,Canada,"March 1, 2020",2019


We need to extract the year in which each tv show/movie was added to Netflix from the `date_added` column

In [6]:
def parse_time(string_time):
    try:
        clean = string_time.strip()
        parsed = datetime.datetime.strptime(clean, '%B %d, %Y')
        return int(parsed.year)

    except Exception as e:
        return 0

In [7]:
expanded_countries['added_year'] = expanded_countries['date_added'].apply(
    parse_time)
with_year = expanded_countries[expanded_countries['added_year'] != 0]

with_year

,title,country,date_added,release_year,added_year
0,3%,Brazil,"August 14, 2020",2020,2020
1,7:19,Mexico,"December 23, 2016",2016,2016
2,23:59,Singapore,"December 20, 2018",2011,2018
3,9,United States,"November 16, 2017",2009,2017
4,21,United States,"January 1, 2020",2008,2020
...,...,...,...,...,...
7784,Zulu Man in Japan,nan,"September 25, 2020",2019,2020
7785,Zumbo's Just Desserts,Australia,"October 31, 2020",2019,2020
7786,ZZ TOP: THAT LITTLE OL' BAND FROM TEXAS,United Kingdom,"March 1, 2020",2019,2020
7786,ZZ TOP: THAT LITTLE OL' BAND FROM TEXAS,Canada,"March 1, 2020",2019,2020


We get all the added_year values available in our dataframe.

In [8]:
years = sorted(expanded_countries['added_year'].unique())

For each year, we count how many shows/movies are produced by each country and create one larger dataframe containing all aggregate data.

In [9]:
counts_by_year_and_country = pd.DataFrame()

for year in years:

    for_year = expanded_countries[
        expanded_countries['added_year'] < year].copy()

    for_year = for_year[['country', 'title']]

    counts_by_country = for_year.groupby(
        by='country', as_index=False).agg(number_of_titles=('title', 'count'))
    counts_by_country['added_year'] = year
    counts_by_year_and_country = pd.concat(
        [counts_by_year_and_country, counts_by_country])

counts_by_year_and_country

,number_of_titles,added_year
0,Australia,2008
1,Japan,2008
2,United Kingdom,2008
3,United States,2008
4,nan,2008
...,...,...
114,Venezuela,2021
115,Vietnam,2021
116,West Germany,2021
117,Zimbabwe,2021


In [10]:
us_movies = counts_by_year_and_country[counts_by_year_and_country['country'] == "United States"]
us_movies

KeyError: 'country'

In [28]:
fig = px.bar(us_movies, x='added_year', y='number_of_titles')
fig.show()

In [29]:
last_year_counts = counts_by_year_and_country[counts_by_year_and_country['added_year'] == 2021]
last_year_counts

,country,number_of_titles,added_year
0,,4,2021
1,Afghanistan,1,2021
2,Albania,1,2021
3,Algeria,2,2021
4,Angola,1,2021
...,...,...,...
114,Venezuela,3,2021
115,Vietnam,5,2021
116,West Germany,5,2021
117,Zimbabwe,3,2021


In [30]:
px.choropleth(last_year_counts,
              locations="country",
              color="number_of_titles",
              hover_name="country",
              hover_data={
                  "country": False,
                  "number_of_titles": True,
                  "added_year": False,
              },
              locationmode='country names',
              color_continuous_scale='reds',
              height=600)

Now we can plot this data on a map and also use the `animation_frame` property to have it variable year.

In [31]:
px.choropleth(counts_by_year_and_country,
              locations="country",
              color="number_of_titles",
              hover_name="country",
              hover_data={
                  "country": False,
                  "number_of_titles": True,
                  "added_year": False,
              },
              locationmode='country names',
              animation_frame="added_year",
              color_continuous_scale='reds',
              height=600)

In [32]:
counts_by_year_and_country[counts_by_year_and_country['country'] != "United States"].country.values

array(['Australia', 'Japan', 'United Kingdom', 'nan', 'Australia',
       'Japan', 'United Kingdom', 'nan', 'Australia', 'Denmark', 'Japan',
       'United Kingdom', 'nan', 'Australia', 'Denmark', 'Japan',
       'United Kingdom', 'nan', 'Australia', 'Denmark', 'France', 'Japan',
       'Mexico', 'Spain', 'United Kingdom', 'nan', 'Australia', 'Denmark',
       'France', 'Japan', 'Mexico', 'Spain', 'United Kingdom', 'nan',
       'Australia', 'Canada', 'Denmark', 'France', 'Ireland', 'Japan',
       'Mexico', 'Spain', 'United Kingdom', 'nan', '', 'Australia',
       'Bermuda', 'Canada', 'Denmark', 'Ecuador', 'Egypt', 'France',
       'Hungary', 'Ireland', 'Japan', 'Mexico', 'Norway', 'Spain',
       'United Kingdom', 'nan', '', 'Australia', 'Belgium', 'Bermuda',
       'Canada', 'China', 'Denmark', 'Ecuador', 'Egypt', 'France',
       'Germany', 'Hungary', 'Ireland', 'Italy', 'Japan', 'Mexico',
       'New Zealand', 'Nigeria', 'Norway', 'Spain', 'Ukraine',
       'United Kingdom', 'nan'

In [33]:
px.choropleth(counts_by_year_and_country[counts_by_year_and_country['country'] != "United States"],
              locations="country",
              color="number_of_titles",
              hover_name="country",
              hover_data={
                  "country": False,
                  "number_of_titles": True,
                  "added_year": False,
              },
              locationmode='country names',
              animation_frame="added_year",
              color_continuous_scale='reds',
              height=600)

### 2. Percentage of TV shows vs movies produced each year

In [13]:
percentages = dataframe.copy()

percentages['added_year'] = percentages['date_added'].apply(
    parse_time)
percentages

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,added_year
0,s1,TV Show,3%,NaN,"João Miguel, Bianca Comparato, Michel Gomes, R...",Brazil,"August 14, 2020",2020,TV-MA,4 Seasons,"International TV Shows, TV Dramas, TV Sci-Fi &...",In a future where the elite inhabit an island ...,2020
1,s2,Movie,7:19,Jorge Michel Grau,"Demián Bichir, Héctor Bonilla, Oscar Serrano, ...",Mexico,"December 23, 2016",2016,TV-MA,93 min,"Dramas, International Movies",After a devastating earthquake hits Mexico Cit...,2016
2,s3,Movie,23:59,Gilbert Chan,"Tedd Chan, Stella Chung, Henley Hii, Lawrence ...",Singapore,"December 20, 2018",2011,R,78 min,"Horror Movies, International Movies","When an army recruit is found dead, his fellow...",2018
3,s4,Movie,9,Shane Acker,"Elijah Wood, John C. Reilly, Jennifer Connelly...",United States,"November 16, 2017",2009,PG-13,80 min,"Action & Adventure, Independent Movies, Sci-Fi...","In a postapocalyptic world, rag-doll robots hi...",2017
4,s5,Movie,21,Robert Luketic,"Jim Sturgess, Kevin Spacey, Kate Bosworth, Aar...",United States,"January 1, 2020",2008,PG-13,123 min,Dramas,A brilliant group of students become card-coun...,2020
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7782,s7783,Movie,Zozo,Josef Fares,"Imad Creidi, Antoinette Turk, Elias Gergi, Car...","Sweden, Czech Republic, United Kingdom, Denmar...","October 19, 2020",2005,TV-MA,99 min,"Dramas, International Movies",When Lebanon's Civil War deprives Zozo of his ...,2020
7783,s7784,Movie,Zubaan,Mozez Singh,"Vicky Kaushal, Sarah-Jane Dias, Raaghav Chanan...",India,"March 2, 2019",2015,TV-14,111 min,"Dramas, International Movies, Music & Musicals",A scrappy but poor boy worms his way into a ty...,2019
7784,s7785,Movie,Zulu Man in Japan,NaN,Nasty C,NaN,"September 25, 2020",2019,TV-MA,44 min,"Documentaries, International Movies, Music & M...","In this documentary, South African rapper Nast...",2020
7785,s7786,TV Show,Zumbo's Just Desserts,NaN,"Adriano Zumbo, Rachel Khoo",Australia,"October 31, 2020",2019,TV-PG,1 Season,"International TV Shows, Reality TV",Dessert wizard Adriano Zumbo looks for the nex...,2020


In [14]:
years = sorted(percentages['added_year'].unique())

In [33]:
counts_by_year_percentage = {"Year": [], "TV Shows %": [], "Movies %": []}

for year in years:

    for_year = percentages[percentages['added_year'] == year]

    shows_count = for_year[for_year['type'] == 'TV Show'].shape[0]
    movies_count = for_year[for_year['type'] == 'Movie'].shape[0]
    shows_percentage = round(shows_count * 100 / for_year.shape[0], 2)
    movies_percentage = round(movies_count * 100 / for_year.shape[0], 2)
    
    counts_by_year_percentage['Year'].append(year)
    counts_by_year_percentage['TV Shows %'].append(shows_percentage)
    counts_by_year_percentage['Movies %'].append(movies_percentage)
    

counts_by_year_percentage= pd.DataFrame(counts_by_year_percentage)
counts_by_year_percentage = counts_by_year_percentage[counts_by_year_percentage['Year'] != 0]

In [34]:
counts_by_year_percentage

,Year,TV Shows %,Movies %
1,2008,50.00,50.00
2,2009,0.00,100.00
3,2010,0.00,100.00
4,2011,0.00,100.00
5,2012,0.00,100.00
6,2013,45.45,54.55
7,2014,24.00,76.00
8,2015,34.09,65.91
9,2016,41.76,58.24
10,2017,29.47,70.53


In [35]:
fig = go.Figure()
fig.add_trace(
    go.Bar(y=counts_by_year_percentage["TV Shows %"],
           x=counts_by_year_percentage.Year,
           name="TV Shows %",
           marker={"color": 'rgba(128,128,64, 0.6)'}))
fig.add_trace(
    go.Bar(y=counts_by_year_percentage["Movies %"],
           x=counts_by_year_percentage.Year,
           name="Movies %",
           marker={"color": 'rgba(255,0,0, 0.6)'}))

fig.update_layout(yaxis={
    "title_text": "Productions added %",
    "ticktext": ["0%", "20%", "40%", "60%", "80%", "100%"],
    "tickvals": [0, 20, 40, 60, 80, 100],
    "tickmode": "array",
    "titlefont": {
        "size": 16
    },
},
                  autosize=False,
                  paper_bgcolor='rgba(0,0,0,0)',
                  plot_bgcolor='rgba(0,0,0,0)',
                  title={
                      'text': "Total Netflix productions %",
                      'y': 0.85,
                      'x': 0.5,
                      'xanchor': 'center',
                      'yanchor': 'top'
                  },
                  barmode='stack')
fig.show()

### 3. Average movie duration

Extract productions that are of type movie

In [39]:
movies = dataframe[dataframe['type'] == 'Movie'].copy()
movies

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
1,s2,Movie,7:19,Jorge Michel Grau,"Demián Bichir, Héctor Bonilla, Oscar Serrano, ...",Mexico,"December 23, 2016",2016,TV-MA,93 min,"Dramas, International Movies",After a devastating earthquake hits Mexico Cit...
2,s3,Movie,23:59,Gilbert Chan,"Tedd Chan, Stella Chung, Henley Hii, Lawrence ...",Singapore,"December 20, 2018",2011,R,78 min,"Horror Movies, International Movies","When an army recruit is found dead, his fellow..."
3,s4,Movie,9,Shane Acker,"Elijah Wood, John C. Reilly, Jennifer Connelly...",United States,"November 16, 2017",2009,PG-13,80 min,"Action & Adventure, Independent Movies, Sci-Fi...","In a postapocalyptic world, rag-doll robots hi..."
4,s5,Movie,21,Robert Luketic,"Jim Sturgess, Kevin Spacey, Kate Bosworth, Aar...",United States,"January 1, 2020",2008,PG-13,123 min,Dramas,A brilliant group of students become card-coun...
6,s7,Movie,122,Yasir Al Yasiri,"Amina Khalil, Ahmed Dawood, Tarek Lotfy, Ahmed...",Egypt,"June 1, 2020",2019,TV-MA,95 min,"Horror Movies, International Movies","After an awful accident, a couple admitted to ..."
...,...,...,...,...,...,...,...,...,...,...,...,...
7781,s7782,Movie,Zoom,Peter Hewitt,"Tim Allen, Courteney Cox, Chevy Chase, Kate Ma...",United States,"January 11, 2020",2006,PG,88 min,"Children & Family Movies, Comedies","Dragged from civilian life, a former superhero..."
7782,s7783,Movie,Zozo,Josef Fares,"Imad Creidi, Antoinette Turk, Elias Gergi, Car...","Sweden, Czech Republic, United Kingdom, Denmar...","October 19, 2020",2005,TV-MA,99 min,"Dramas, International Movies",When Lebanon's Civil War deprives Zozo of his ...
7783,s7784,Movie,Zubaan,Mozez Singh,"Vicky Kaushal, Sarah-Jane Dias, Raaghav Chanan...",India,"March 2, 2019",2015,TV-14,111 min,"Dramas, International Movies, Music & Musicals",A scrappy but poor boy worms his way into a ty...
7784,s7785,Movie,Zulu Man in Japan,NaN,Nasty C,NaN,"September 25, 2020",2019,TV-MA,44 min,"Documentaries, International Movies, Music & M...","In this documentary, South African rapper Nast..."


Convert the duration column from string `x min` to the number `x`

In [40]:
movies['duration'] = movies['duration'].apply(lambda x: int(x.split()[0]))

movies

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
1,s2,Movie,7:19,Jorge Michel Grau,"Demián Bichir, Héctor Bonilla, Oscar Serrano, ...",Mexico,"December 23, 2016",2016,TV-MA,93,"Dramas, International Movies",After a devastating earthquake hits Mexico Cit...
2,s3,Movie,23:59,Gilbert Chan,"Tedd Chan, Stella Chung, Henley Hii, Lawrence ...",Singapore,"December 20, 2018",2011,R,78,"Horror Movies, International Movies","When an army recruit is found dead, his fellow..."
3,s4,Movie,9,Shane Acker,"Elijah Wood, John C. Reilly, Jennifer Connelly...",United States,"November 16, 2017",2009,PG-13,80,"Action & Adventure, Independent Movies, Sci-Fi...","In a postapocalyptic world, rag-doll robots hi..."
4,s5,Movie,21,Robert Luketic,"Jim Sturgess, Kevin Spacey, Kate Bosworth, Aar...",United States,"January 1, 2020",2008,PG-13,123,Dramas,A brilliant group of students become card-coun...
6,s7,Movie,122,Yasir Al Yasiri,"Amina Khalil, Ahmed Dawood, Tarek Lotfy, Ahmed...",Egypt,"June 1, 2020",2019,TV-MA,95,"Horror Movies, International Movies","After an awful accident, a couple admitted to ..."
...,...,...,...,...,...,...,...,...,...,...,...,...
7781,s7782,Movie,Zoom,Peter Hewitt,"Tim Allen, Courteney Cox, Chevy Chase, Kate Ma...",United States,"January 11, 2020",2006,PG,88,"Children & Family Movies, Comedies","Dragged from civilian life, a former superhero..."
7782,s7783,Movie,Zozo,Josef Fares,"Imad Creidi, Antoinette Turk, Elias Gergi, Car...","Sweden, Czech Republic, United Kingdom, Denmar...","October 19, 2020",2005,TV-MA,99,"Dramas, International Movies",When Lebanon's Civil War deprives Zozo of his ...
7783,s7784,Movie,Zubaan,Mozez Singh,"Vicky Kaushal, Sarah-Jane Dias, Raaghav Chanan...",India,"March 2, 2019",2015,TV-14,111,"Dramas, International Movies, Music & Musicals",A scrappy but poor boy worms his way into a ty...
7784,s7785,Movie,Zulu Man in Japan,NaN,Nasty C,NaN,"September 25, 2020",2019,TV-MA,44,"Documentaries, International Movies, Music & M...","In this documentary, South African rapper Nast..."


Group movies by their release year and compute average, minimum and maximum durations

In [41]:
duration_per_year = movies.groupby(by='release_year', as_index=False).agg(
    num_movies=('show_id', 'count'),
    average_duration=('duration', 'mean'),
    min_duration=('duration', 'min'),
    max_duration=('duration', 'max'))
duration_per_year

,release_year,num_movies,average_duration,min_duration,max_duration
0,1942,2,35.000000,18,52
1,1943,3,62.666667,45,82
2,1944,3,52.000000,40,76
3,1945,3,51.333333,32,63
4,1946,1,58.000000,58,58
...,...,...,...,...,...
67,2017,744,94.836022,11,166
68,2018,734,96.170300,12,312
69,2019,582,93.457045,10,209
70,2020,411,89.795620,5,190


Let's see the number of movies produced every year

In [42]:
fig = px.bar(duration_per_year, x='release_year', y='num_movies')
fig.show()

It looks like most of the movies are released after year 2000 so we will only use these ones in oroder to have correct metrics  

In [43]:
recent_movies = duration_per_year[duration_per_year['release_year'] > 2000].copy()

recent_movies

,release_year,num_movies,average_duration,min_duration,max_duration
51,2001,32,118.875000,42,224
52,2002,39,118.871795,78,179
53,2003,40,117.125000,37,201
54,2004,50,115.940000,46,185
55,2005,61,114.360656,55,208
56,2006,72,114.541667,45,196
57,2007,72,113.069444,70,169
58,2008,102,106.666667,25,214
59,2009,103,108.621359,29,203
60,2010,136,104.426471,23,186


We can also compute the regression line of the average duration

In [75]:
model.kernel

'rbf'

In [80]:
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler


x = recent_movies['release_year'].values.reshape(-1, 1)
y = recent_movies['average_duration'].values

model = make_pipeline(StandardScaler(), SVR())

model = SVR(kernel='linear')
model.fit(x, y)

x_future = np.linspace(2021, 2025, 100)
y_future = model.predict(x_future.reshape(-1, 1))

Plot all the data

In [85]:
fig = go.Figure()

fig.add_trace(
    go.Scatter(x=recent_movies['release_year'],
               y=recent_movies['average_duration'],
               name='Average duration',
               mode='lines+markers',
               line={
                   "color": 'firebrick',
                   "width": 2,
               }))
fig.add_trace(
    go.Scatter(x=recent_movies['release_year'],
               y=recent_movies['min_duration'],
               name='Minimum duration',
               mode='lines+markers',
               line={
                   "color": 'royalblue',
                   "width": 2,
                   "dash": 'dot'
               }))

fig.add_traces(
    go.Scatter(x=x_future,
               y=y_future.reshape(-1, ),
               name='Regression Line',
               line={
                   "color": 'blueviolet',
                   "width": 2,
                   "dash":"dashdot"
               }))
fig.add_trace(
    go.Scatter(x=recent_movies['release_year'],
               y=recent_movies['max_duration'],
               name='Maximum duration',
               mode='lines+markers',
               line={
                   "color": 'green',
                   "width": 2,
                   "dash": 'dot'
               }))

fig.update_layout(
    title='Duration of movies from the Netflix collection by release year',
    xaxis_title='Release year',
    yaxis_title='Duration (minutes)')

fig.show()

In [82]:
model.coef_

array([[-1.60635561]])

In [51]:
recent_movies['average_duration'].values

array([118.875     , 118.87179487, 117.125     , 115.94      ,
       114.36065574, 114.54166667, 113.06944444, 106.66666667,
       108.62135922, 104.42647059, 102.82677165, 100.62025316,
        98.75247525, 100.26639344,  99.82722513,  95.11214953,
        94.83602151,  96.17029973,  93.45704467,  89.79562044,
        80.16666667])

In [ ]:
recent_movies